In [1]:
%load_ext autoreload
%autoreload 2

## Dataset & Dataloader

In [2]:
import os
os.chdir("../")

from omegaconf import OmegaConf

from music_embedding.data.datasets import MusicSegmentDataset
from music_embedding.utils.common import load_dataset

In [12]:
config = "configs/train.yaml"
config = OmegaConf.load(config)

dataset = MusicSegmentDataset(config["dataset"]["train"])

Loading Background noise:   0%|          | 0/2142 [00:00<?, ?it/s]


RuntimeError: Failed to open the input "aug/bg/tr/random_noise/pink/475.wav" (No such file or directory).
Exception raised from get_input_format_context at /__w/_temp/conda_environment_8616573414/conda-bld/torchaudio_1712669598216/work/src/libtorio/ffmpeg/stream_reader/stream_reader.cpp:42 (most recent call first):
frame #0: c10::Error::Error(c10::SourceLocation, std::string) + 0x57 (0x7bc9b81ee897 in /home/huynd/anaconda3/envs/audiofp/lib/python3.10/site-packages/torch/lib/libc10.so)
frame #1: c10::detail::torchCheckFail(char const*, char const*, unsigned int, std::string const&) + 0x64 (0x7bc9b819eb25 in /home/huynd/anaconda3/envs/audiofp/lib/python3.10/site-packages/torch/lib/libc10.so)
frame #2: <unknown function> + 0x42334 (0x7bc9a0e6e334 in /home/huynd/anaconda3/envs/audiofp/lib/python3.10/site-packages/torio/lib/libtorio_ffmpeg4.so)
frame #3: torio::io::StreamingMediaDecoder::StreamingMediaDecoder(std::string const&, std::optional<std::string> const&, std::optional<std::map<std: